In [1]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from transformers import Wav2Vec2Model
from audio_dataset import get_data_loaders

audio_encoder = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

c:\Users\gnith\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
from audio_dataset import get_data_loaders

audio_dir = r"C:\Users\gnith\Desktop\multi_modal_audio_speech\ravdess_dataset"

    # Get train and validation dataloaders
train_loader, val_loader = get_data_loaders(audio_dir, batch_size=2)

In [3]:
for a in train_loader:
    print(a)
    break

{'input_values': tensor([[-7.0262e-03, -8.2524e-03, -1.0697e-02,  ..., -7.0473e-03,
         -1.2591e-01, -2.1741e-01],
        [-3.1370e-06, -1.3217e-05, -9.6774e-06,  ..., -1.0263e-05,
         -1.1941e-05, -2.2046e-06]]), 'attention_mask': None, 'emotions': tensor([2, 6]), 'metadata': ({'modality': 3, 'vocal_channel': 2, 'emotion': 2, 'intensity': 2, 'statement': 1, 'repetition': 1, 'gender': 0}, {'modality': 3, 'vocal_channel': 1, 'emotion': 6, 'intensity': 2, 'statement': 2, 'repetition': 2, 'gender': 1})}


In [9]:
audio_encoder(a['input_values']).last_hidden_state.shape

torch.Size([2, 149, 768])

In [13]:
audio_encoder(a['input_values']).extract_features.mean(dim=1).shape

torch.Size([2, 512])

In [15]:
train_loader, test_loader = get_data_loaders("ravdess_dataset", batch_size=2, augment=False)

In [2]:
import timm

clip = timm.create_model(
            'vit_base_patch16_clip_224.openai',
            pretrained=True,
            num_classes=0,  # We'll add our own head
            in_chans=3  # Our spectrograms have 3 channels
        )

c:\Users\gnith\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gnith\.cache\huggingface\hub\models--timm--vit_base_patch16_clip_224.openai. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:
clip

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): 

In [5]:
for a,b in vit.named_parameters():
    print(a)

cls_token
pos_embed
patch_embed.proj.weight
patch_embed.proj.bias
blocks.0.norm1.weight
blocks.0.norm1.bias
blocks.0.attn.qkv.weight
blocks.0.attn.qkv.bias
blocks.0.attn.proj.weight
blocks.0.attn.proj.bias
blocks.0.norm2.weight
blocks.0.norm2.bias
blocks.0.mlp.fc1.weight
blocks.0.mlp.fc1.bias
blocks.0.mlp.fc2.weight
blocks.0.mlp.fc2.bias
blocks.1.norm1.weight
blocks.1.norm1.bias
blocks.1.attn.qkv.weight
blocks.1.attn.qkv.bias
blocks.1.attn.proj.weight
blocks.1.attn.proj.bias
blocks.1.norm2.weight
blocks.1.norm2.bias
blocks.1.mlp.fc1.weight
blocks.1.mlp.fc1.bias
blocks.1.mlp.fc2.weight
blocks.1.mlp.fc2.bias
blocks.2.norm1.weight
blocks.2.norm1.bias
blocks.2.attn.qkv.weight
blocks.2.attn.qkv.bias
blocks.2.attn.proj.weight
blocks.2.attn.proj.bias
blocks.2.norm2.weight
blocks.2.norm2.bias
blocks.2.mlp.fc1.weight
blocks.2.mlp.fc1.bias
blocks.2.mlp.fc2.weight
blocks.2.mlp.fc2.bias
blocks.3.norm1.weight
blocks.3.norm1.bias
blocks.3.attn.qkv.weight
blocks.3.attn.qkv.bias
blocks.3.attn.proj.wei

In [19]:
import pickle

# Replace 'file_path' with the actual path to your .pickle file
with open('train_data.pickle', 'rb') as f:
    data = pickle.load(f)


In [24]:
res_1 = data[0]['images']
res_1.shape

(120, 64, 64)

In [29]:
import torch.nn.functional as F

res_2 = torch.tensor(res_1)
resized_images = F.interpolate(res_2.unsqueeze(1), size=(224, 224), mode='bilinear', align_corners=False).squeeze(1)
resized_images.shape


torch.Size([120, 224, 224])

In [16]:
import torch

import torch.nn.functional as F

# Assuming the images are stored in a tensor
images_tensor = torch.rand(120,1,64, 64)  # Shape: (120, 1, 64, 64)

# Resize the images
resized_images = F.interpolate(images_tensor, size=(224, 224), mode='bilinear', align_corners=False)

# Update the data list with resized images
for i, item in enumerate(data):
    item['images'] = resized_images[i]

In [17]:
resized_images.shape

torch.Size([120, 1, 224, 224])

In [1]:
import pickle
import cv2

# Replace 'file_path' with the actual path to your .pickle file
with open('multimodal_dataset_temp.pickle', 'rb') as f:
    data = pickle.load(f)


In [ ]:
# import pickle
# import cv2

# # Replace 'file_path' with the actual path to your .pickle file
# with open('multimodal_dataset_temp.pickle', 'rb') as f:
#     data = pickle.load(f)

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedShuffleSplit
# # Ensure uniform distribution of the 'emotion' labels
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

# # Split into train and temp sets
# for train_idx, temp_idx in splitter.split(data, [item['emotion'] for item in data]):
#     train_data = [data[i] for i in train_idx]
#     temp_data = [data[i] for i in temp_idx]

# # Further split temp into validation and test sets
# splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)
# for val_idx, test_idx in splitter.split(temp_data, [item['emotion'] for item in temp_data]):
#     val_data = [temp_data[i] for i in val_idx]
#     test_data = [temp_data[i] for i in test_idx]

# # Print the sizes of each split
# print(f"Train size: {len(train_data)}")
# print(f"Validation size: {len(val_data)}")
# print(f"Test size: {len(test_data)}")


# from torch.utils.data import Dataset, DataLoader
# import torch
# import torch.nn.functional as F

# class MultimodalDataset(Dataset):
#     def __init__(self, data):
#         self.data = data

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         audio = F.interpolate(torch.tensor(item['audio'], dtype=torch.float32).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
#         audio = audio.repeat(3, 1, 1)  # Repeat the audio into 3 channels

#         #considering only the 10 frames due to size difference
#         video = F.interpolate(torch.tensor(item['video'][:10], dtype=torch.float32).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
#         text = item['text']  # Assuming text is a string
#         emotion = torch.tensor(item['emotion'], dtype=torch.long)
#         return {'audio': audio, 'video': video, 'text': text, 'emotion': emotion}

# def get_dataloader(data, batch_size=32, shuffle=True):
#     dataset = MultimodalDataset(data)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
#     return dataloader


Train size: 1716
Validation size: 368
Test size: 368


In [ ]:
# from torch.utils.data import Dataset, DataLoader
# import torch
# import torch.nn.functional as F

# class MultimodalDataset(Dataset):
#     def __init__(self, data):
#         self.data = data

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         item = self.data[idx]
#         audio = F.interpolate(torch.tensor(item['audio'], dtype=torch.float32).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
#         audio = audio.repeat(3, 1, 1)  # Repeat the audio into 3 channels

#         #considering only the 10 frames due to size difference
#         video = F.interpolate(torch.tensor(item['video'][:10], dtype=torch.float32).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
#         text = item['text']  # Assuming text is a string
#         emotion = torch.tensor(item['emotion'], dtype=torch.long)
#         return {'audio': audio, 'video': video, 'text': text, 'emotion': emotion}

# def get_dataloader(data, batch_size=32, shuffle=True):
#     dataset = MultimodalDataset(data)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
#     return dataloader

# # Example usage:
# train_dataloader = get_dataloader(train_data, batch_size=2, shuffle=True)
# val_dataloader = get_dataloader(val_data, batch_size=2, shuffle=False)
# test_dataloader = get_dataloader(test_data, batch_size=2, shuffle=False)

In [10]:
import pickle
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter
import numpy as np

class MultimodalDataProcessor:
    def __init__(self, pickle_file, batch_size=32, test_size=0.3, val_size=0.5, random_state=42):
        self.pickle_file = pickle_file
        self.batch_size = batch_size
        self.test_size = test_size
        self.val_size = val_size
        self.random_state = random_state
        self.train_data = None
        self.val_data = None
        self.test_data = None
        self.class_weights = None

    def load_data(self):
        with open(self.pickle_file, 'rb') as f:
            self.data = pickle.load(f)

    def split_data(self):
        # Ensure uniform distribution of the 'emotion' labels
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=self.test_size, random_state=self.random_state)

        # Split into train and temp sets
        for train_idx, temp_idx in splitter.split(self.data, [item['emotion'] for item in self.data]):
            self.train_data = [self.data[i] for i in train_idx]
            temp_data = [self.data[i] for i in temp_idx]

        # Further split temp into validation and test sets
        splitter = StratifiedShuffleSplit(n_splits=1, test_size=self.val_size, random_state=self.random_state)
        for val_idx, test_idx in splitter.split(temp_data, [item['emotion'] for item in temp_data]):
            self.val_data = [temp_data[i] for i in val_idx]
            self.test_data = [temp_data[i] for i in test_idx]

    def compute_class_weights(self):
        # Compute class weights for cross-entropy loss
        emotion_counts = Counter([item['emotion'] for item in self.train_data])
        total_samples = len(self.train_data)
        self.class_weights = torch.tensor(
            [total_samples / (len(emotion_counts) * emotion_counts[i]) for i in range(len(emotion_counts))],
            dtype=torch.float32
        )

    class MultimodalDataset(Dataset):
        def __init__(self, data):
            self.data = data

        def __len__(self):
            return len(self.data)

        def __getitem__(self, idx):
            item = self.data[idx]
            audio = F.interpolate(torch.tensor(item['audio'], dtype=torch.float32).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
            audio = audio.repeat(3, 1, 1)  # Repeat the audio into 3 channels

            # Considering only the first 10 frames due to size difference
            video = F.interpolate(torch.tensor(item['video'][:10], dtype=torch.float32).unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False).squeeze(0)
            text = item['text']  # Assuming text is a string
            emotion = torch.tensor(item['emotion'], dtype=torch.long)
            return {'audio': audio, 'video': video, 'text': text, 'emotion': emotion}

    def get_dataloader(self, data, shuffle=True):
        dataset = self.MultimodalDataset(data)
        dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=shuffle)
        return dataloader

    def process(self):
        self.load_data()
        self.split_data()
        self.compute_class_weights()
        train_dataloader = self.get_dataloader(self.train_data, shuffle=True)
        val_dataloader = self.get_dataloader(self.val_data, shuffle=False)
        test_dataloader = self.get_dataloader(self.test_data, shuffle=False)
        return train_dataloader, val_dataloader, test_dataloader, self.class_weights


multimodel_dataloader = MultimodalDataProcessor("multimodal_dataset_temp.pickle", batch_size=2)
train_dataloader, val_dataloader, test_dataloader, class_weights = multimodel_dataloader.process()

tensor([[ 0.1490,  0.2664, -0.2436,  0.1619, -0.1191,  0.0543, -0.4196,  0.3735],
        [-0.1723,  0.1628,  0.1190,  0.0502,  0.0450,  0.1518, -0.4038,  0.3089]],
       grad_fn=<AddmmBackward0>) tensor([2, 3])
tensor(2.2318, grad_fn=<NllLossBackward0>)


In [ ]:

import torch
import timm
import torch.nn as nn
import pytorch_lightning as pl
from torch.utils.data import DataLoader
from transformers import HubertModel
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from peft import get_peft_model, LoraConfig, TaskType  # Import PEFT components


class ImageEmotionClassifier(pl.LightningModule):
    def __init__(self, num_classes, model_name='vit_base_patch16_224', pretrained=True):
        super(ImageEmotionClassifier, self).__init__()
        
        # Load ViT model from timm
        self.vit = timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=8,  # We'll add our own head
            in_chans=10  # Video data has 0 input channels
        )
        
        # Get the feature dimension of the ViT
        self.feature_dim = self.vit.num_features
        
        # Classifier head
        self.classifier = nn.Sequential(
            nn.LayerNorm(self.feature_dim),
            nn.Linear(self.feature_dim, 512),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(512, num_classes)
        )
        
        # Initialize weights for classifier
        self._init_weights(self.classifier)

    def _init_weights(self, module):
        """Initialize weights for linear layers"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)

    def forward(self, x):
        # Get features from ViT
        features = self.vit(x)
        
        # Classify
        logits = self.classifier(features)
        return logits

    def training_step(self, batch, batch_idx):
        video = batch['video']
        labels = batch['emotion']
        logits = self(video)
        print(logits,labels)
        loss = nn.CrossEntropyLoss()(logits, labels)
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        
        self.log("train_loss", loss, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        video = batch['video']
        labels = batch['emotion']
        logits = self(video)
        loss = nn.CrossEntropyLoss()(logits, labels)
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=1)
        acc = (preds == labels).float().mean()
        
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        # Separate parameters for the ViT and classifier
        vit_params = []
        classifier_params = []
        
        for name, param in self.named_parameters():
            if 'vit' in name:
                if 'mlp' in name:
                    vit_params.append(param)
                else:
                    param.requires_grad = False
            else:
                classifier_params.append(param)
        
        optimizer = torch.optim.AdamW([
            {'params': vit_params, 'lr': 1e-5},  # Lower learning rate for pretrained ViT
            {'params': classifier_params, 'lr': 1e-4}  # Higher learning rate for new head
        ], weight_decay=0.01)
        
        # Add learning rate scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
        
        return [optimizer], [scheduler]


In [7]:
model = ImageEmotionClassifier(num_classes=0)
trainer = pl.Trainer(max_epochs=5, devices=1)
trainer.fit(model, train_dataloader, val_dataloader)

c:\Users\gnith\miniconda3\Lib\site-packages\torch\nn\init.py:511: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type              | Params | Mode 
---------------------------------------------------------
0 | vit        | VisionTransformer 

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\gnith\miniconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
train_data